In [98]:
# Imports
import json, os, re, shutil, sys, time
import seaborn as sns
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML
from collections import defaultdict
import xmltodict
import untangle
import xml.etree.ElementTree as ET
# NLTK for NLP utils and corpora
import nltk
from nltk.corpus import treebank
from nltk.text import Text
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

# Import spacy
import spacy

import sklearn
from sklearn.model_selection import train_test_split

# NumPy, Pandas and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf

from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata

from numpy.random import seed
from pandas import read_csv, DataFrame
from sklearn.preprocessing import minmax_scale

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.embeddings import Embedding
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Activation, Dropout, Conv1D, MaxPooling1D, Bidirectional, Flatten, TimeDistributed

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

## AEG Long Essay

In [2]:
aeg_long = pd.read_csv("../data-DNC/AEG/training_set_rel3.tsv",sep='\t',encoding = "latin1")
#aeg_long.columns = aeg_long.iloc[0]
#aeg_long.reindex(aeg_long.index.drop(0))
aeg_long.head(5)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def get_score(essay):
    score = 0
    score = float(essay[3])
    essay_set = essay[1]
    if essay_set == 1:
        div = 12
    elif essay_set == 2:
        div = 5
    elif essay_set == 3:
        div = 3
    elif essay_set == 4:
        div = 3
    elif essay_set == 5:
        div = 4
    elif essay_set == 6:
        div = 4
    elif essay_set == 7:
        div = 25
    elif essay_set == 8:
        div = 50
    return score/div

In [4]:
nlp = spacy.load('en_core_web_lg')

In [7]:
sen = "I am good. I need to be better. Are you good? You are awesome! Let's stop here."
tk = nlp(sen)
for s in tk.sents:
    print(s)

I am good.
I need to be better.
Are you good?
You are awesome!
Let's stop here.


In [5]:
aeg_long_mod = aeg_long[['essay_id','essay_set','essay','domain1_score']]

In [6]:
from datetime import datetime
start=datetime.now()
aeg_long_sentence = pd.DataFrame(columns=['essay_id','essay_set','sentence','domain1_score'])
for i in range(len(aeg_long_mod)):
    if i%1000 == 0:
        print("At iteration :",i)
        print("Duration: ",datetime.now()-start)
    sentence = nlp(aeg_long_mod.essay[i])
    for s in sentence.sents:
        aeg_long_sentence = aeg_long_sentence.append({'essay_id' : aeg_long_mod.essay_id[i],
                                                      'essay_set' : aeg_long_mod.essay_set[i],'sentence' : s, 
                                                      'domain1_score' : aeg_long_mod.domain1_score[i]},
                                                     ignore_index=True)

At iteration : 0
Duration:  0:00:00.001225
At iteration : 1000
Duration:  0:02:26.681574
At iteration : 2000
Duration:  0:05:59.845798
At iteration : 3000
Duration:  0:11:11.224126
At iteration : 4000
Duration:  0:15:46.331055
At iteration : 5000
Duration:  0:17:50.410508
At iteration : 6000
Duration:  0:19:45.485454
At iteration : 7000
Duration:  0:21:39.620808
At iteration : 8000
Duration:  0:24:15.281349
At iteration : 9000
Duration:  0:27:08.179534
At iteration : 10000
Duration:  0:30:36.951509
At iteration : 11000
Duration:  0:35:13.829451
At iteration : 12000
Duration:  0:42:56.885765


In [7]:
aeg_long_sentence.head(30)

,essay_id,essay_set,sentence,domain1_score
0,1,1,"(Dear, local, newspaper, ,, I, think, effects,...",8
1,1,1,"(Thing, about, !)",8
2,1,1,"(Do, nt, you, think, so, ?)",8
3,1,1,"(How, would, you, feel, if, your, teenager, is...",8
4,1,1,"(Do, you, ever, time, to, chat, with, your, fr...",8
5,1,1,"(Well, now, -, there, 's, a, new, way, to, cha...",8
6,1,1,"(Just, think, now, while, your, setting, up, m...",8
7,1,1,"(How, did, you, learn, about, other, countrys,...",8
8,1,1,"(Well, I, have, by, computer, /, internet)",8
9,1,1,"(,, it, 's, a, new, way, to, learn, about, wha...",8


In [8]:
aeg_long_sentence['Norm_Score'] = aeg_long_sentence.apply(get_score,axis=1)

In [9]:
def spacy_to_text(essay):
    return essay[2].text

In [10]:
aeg_long_sentence['Mod_Sentence'] = aeg_long_sentence.apply(spacy_to_text,axis=1)

In [11]:
aeg_long_sentence.head(30)

,essay_id,essay_set,sentence,domain1_score,Norm_Score,Mod_Sentence
0,1,1,"(Dear, local, newspaper, ,, I, think, effects,...",8,0.666667,"Dear local newspaper, I think effects computer..."
1,1,1,"(Thing, about, !)",8,0.666667,Thing about!
2,1,1,"(Do, nt, you, think, so, ?)",8,0.666667,Dont you think so?
3,1,1,"(How, would, you, feel, if, your, teenager, is...",8,0.666667,How would you feel if your teenager is always ...
4,1,1,"(Do, you, ever, time, to, chat, with, your, fr...",8,0.666667,Do you ever time to chat with your friends or ...
5,1,1,"(Well, now, -, there, 's, a, new, way, to, cha...",8,0.666667,Well now - there's a new way to chat the compu...
6,1,1,"(Just, think, now, while, your, setting, up, m...",8,0.666667,Just think now while your setting up meeting w...
7,1,1,"(How, did, you, learn, about, other, countrys,...",8,0.666667,How did you learn about other countrys/states ...
8,1,1,"(Well, I, have, by, computer, /, internet)",8,0.666667,Well I have by computer/internet
9,1,1,"(,, it, 's, a, new, way, to, learn, about, wha...",8,0.666667,", it's a new way to learn about what going on ..."


In [12]:
train_x_sentence,test_x_sentence,train_y_sentence,test_y_sentence = train_test_split(np.asarray(aeg_long_sentence.Mod_Sentence), np.asarray(aeg_long_sentence.Norm_Score),test_size=0.2)

In [121]:
train_x_sentence,test_x_sentence,train_y_orig_score,test_y_orig_score,train_y_sentence,test_y_sentence = train_test_split(np.asarray(aeg_long_sentence.Mod_Sentence),np.asarray(aeg_long_sentence.domain1_score), np.asarray(aeg_long_sentence.Norm_Score),test_size=0.2)

In [124]:
print(train_y_orig_score.shape)
print(test_y_orig_score.shape)
print(train_y_orig_score[:4])

(147120,)
(36781,)
[46 3 4 32]


In [125]:
vocabulary_size = 50000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_x_sentence)
train_seq = tokenizer.texts_to_sequences(train_x_sentence)
train_data = pad_sequences(train_seq)

In [126]:
max_len_class = train_data.shape[1]
max_words_class = vocabulary_size

In [127]:
test_seq = tokenizer.texts_to_sequences(test_x_sentence)
test_data = pad_sequences(test_seq, maxlen=max_len_class)
test_data.shape

(36781, 175)

In [128]:
aeg_long_sentence.shape

(183901, 6)

In [129]:
embeddings_index = dict()
f = open('/Users/kurapati/W266/data/glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1917494 word vectors.


In [130]:
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [16]:
def RMSE(actual, predict):
    diff = actual - predict
    diff = sum(diff**2) / len(actual)
    return np.sqrt(diff)

In [56]:
#Cohen Kappa score as defined by the kaggle challenge/wikipedia
def CohenKappa(actual, predict):
    CohenDF = pd.DataFrame([actual.astype(np.double).round(), np.around(predict.astype(np.double))]).transpose()
    count = len(CohenDF)
    CohenDF.columns = ['actual','predict']
    correct = len(CohenDF[CohenDF.actual==CohenDF.predict])
    acc = correct / count
    pe = 0
    for value in CohenDF.actual.unique():
        pe += len(CohenDF[CohenDF.actual == value]) * len(CohenDF[CohenDF.predict == value])
    pe = pe / np.square(count)
    return(1 - (1-acc)/(1-pe))

In [63]:
import skll.metrics as metric

# Calculates the average quadratic kappa for the entire essay set
def get_average_kappa(targets, predictions):
        num = len(targets)
        total_kappa = 0

        for i in range(0, num):
                total_kappa += metric.kappa([targets], [predictions], 'quadratic')

        avg_kappa  = float(total_kappa) / float(num)
        return avg_kappa

In [78]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [131]:
tfidf_all = TfidfVectorizer()
tfidf_all.fit(train_x_sentence)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [132]:
train_transform = tfidf_all.transform(train_x_sentence)
test_transform = tfidf_all.transform(test_x_sentence)

In [23]:
gnb = GaussianNB()
lr = LinearRegression(fit_intercept=True)
rf = RandomForestClassifier(n_estimators = 100)

In [ ]:
gnb.fit(train_transform.toarray(), train_y_sentence)
lr.fit(train_transform.toarray(), train_y_sentence)
rf.fit(train_transform.toarray(), train_y_sentence)

In [81]:
train_transform.shape

(147120, 34007)

In [133]:
max_len_class = train_data.shape[1]
max_words_class = vocabulary_size

In [134]:
# For custom metrics
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

#model definitions
def CNN():
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class,300,input_length=max_len_class))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.LSTM(100))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model_conv.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_conv

def CNN_glove_1():
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.LSTM(100))
    model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    #sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    #model_conv.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_conv

def stack_lstm():
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model_conv.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model_conv.add(tf.keras.layers.LSTM(32))
    model_conv.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))
    #sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    model_conv.compile(loss = 'mse', optimizer = tf.train.AdamOptimizer(), metrics = ['accuracy'])
    #model_conv.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_conv
def CNN_glove():
    conv_glove = Sequential()
    conv_glove.add(Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    conv_glove.add(Conv1D(64, 5, activation = 'relu'))
    conv_glove.add(MaxPooling1D(4))
    conv_glove.add(LSTM(100))
    conv_glove.add(Dense(100))
    conv_glove.add(Dense(1, kernel_initializer='normal'))
    sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    conv_glove.compile(loss = 'mse', optimizer = sgd, metrics = ['accuracy'])
    return conv_glove

def CNN_no_dense():
    model_conv = Sequential()
    model_conv.add(tf.keras.layers.Embedding(max_words_class, 300, input_length=max_len_class, weights=[embedding_matrix], trainable=False))
    #model_conv.add(tf.keras.layers.Embedding(max_words_class,300,input_length=max_len_class))
    model_conv.add(tf.keras.layers.Dropout(0.1))
    model_conv.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
    model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=4))
    model_conv.add(tf.keras.layers.LSTM(100))
    #model_conv.add(tf.keras.layers.Dense(100))
    model_conv.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model_conv.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_conv

#bidirectional RNN is screwed up atm
def Bidirectional_RNN():
    inputs = Input(name='inputs',shape=[max_len_class])
    layer = Embedding(max_words,200,input_length=max_len_class)(inputs)
    layer = Bidirectional(LSTM(10))(layer)
    layer = Dense(32,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

def feedforward_NN():
    model_ff = Sequential()
    model_ff.add(tf.keras.layers.Embedding(max_words_class,200,input_length=max_len_class))
    model_ff.add(tf.keras.layers.Flatten())
    model_ff.add(tf.keras.layers.Dense(50,activation='tanh'))
    model_ff.add(tf.keras.layers.Dense(1,activation='sigmoid'))
    model_ff.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_ff

def RNN():
    model_rnn = Sequential()
    model_rnn.add(tf.keras.layers.Embedding(max_words_class,200,input_length=max_len_class))
    model_rnn.add(tf.keras.layers.LSTM(10))
    model_rnn.add(tf.keras.layers.Dense(1,name='out_layer'))
    model_rnn.compile(optimizer=tf.train.AdamOptimizer(),loss='mse',metrics=['accuracy'])
    return model_rnn

In [27]:
feedforward = feedforward_NN()
feedforward.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 175, 200)          10000000  
_________________________________________________________________
flatten (Flatten)            (None, 35000)             0         
_________________________________________________________________
dense (Dense)                (None, 50)                1750050   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 11,750,101
Trainable params: 11,750,101
Non-trainable params: 0
_________________________________________________________________


In [96]:
CNN_glove_model = CNN_glove_1()
CNN_glove_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 175, 300)          15000000  
_________________________________________________________________
dropout_2 (Dropout)          (None, 175, 300)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 171, 64)           96064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 42, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total para

In [135]:
estimator = KerasRegressor(build_fn=CNN_glove_1, epochs=10, batch_size=50)
#kfold = KFold(n_splits=5, random_state=43)
#results = np.sqrt(-1*cross_val_score(estimator, train_data, train_y_sentence,scoring= "neg_mean_squared_error", cv=kfold))
#print("Training RMSE mean and std from CV: {} {}".format(results.mean(),results.std()))

In [136]:
estimator.fit(train_data, train_y_sentence)
prediction_cnn_glove=estimator.predict(test_data)
rmse_val = RMSE(test_y_sentence,prediction_cnn_glove)
print("RMSE: ",rmse_val)

Epoch 1/10
147120/147120 [==============================] - 309s 2ms/step - loss: 0.0594 - acc: 0.0978
Epoch 2/10
147120/147120 [==============================] - 303s 2ms/step - loss: 0.0301 - acc: 0.0991
Epoch 3/10
147120/147120 [==============================] - 282s 2ms/step - loss: 0.0302 - acc: 0.0993
Epoch 4/10
147120/147120 [==============================] - 287s 2ms/step - loss: 0.0296 - acc: 0.0994
Epoch 5/10
147120/147120 [==============================] - 313s 2ms/step - loss: 0.0287 - acc: 0.0999
Epoch 6/10
147120/147120 [==============================] - 319s 2ms/step - loss: 0.0280 - acc: 0.1003
Epoch 7/10
147120/147120 [==============================] - 304s 2ms/step - loss: 0.0275 - acc: 0.1005
Epoch 8/10
147120/147120 [==============================] - 319s 2ms/step - loss: 0.0268 - acc: 0.1008
Epoch 9/10
147120/147120 [==============================] - 321s 2ms/step - loss: 0.0262 - acc: 0.1010
Epoch 10/10
147120/147120 [==============================] - 317s 2ms/ste

In [143]:
print(prediction_cnn_glove.shape)
print(test_y_orig_score.shape)

(36781,)
(36781,)


In [144]:
def find_mult_factor(x):
    if x[1] == 0:
        return 0
    else:
        return np.around(x[0]/x[1])
cnn_df = pd.DataFrame([test_y_orig_score.astype(np.double), test_y_sentence.astype(np.double),
                      prediction_cnn_glove.astype(np.double)]).transpose()
cnn_df.columns = ['Orig_Score','Norm_Score','Pred_Score']
cnn_df.head(10)

,Orig_Score,Norm_Score,Pred_Score
0,5.0,1.000000,0.769634
1,0.0,0.000000,0.474710
2,8.0,0.666667,0.728527
3,13.0,0.520000,0.757053
4,11.0,0.916667,0.711505
5,0.0,0.000000,0.882032
6,21.0,0.840000,0.773958
7,8.0,0.666667,0.815720
8,43.0,0.860000,0.747872
9,6.0,0.500000,0.672615


In [145]:
cnn_df['Mult_Factor'] = cnn_df.apply(find_mult_factor,axis=1)
cnn_df.head(10)

,Orig_Score,Norm_Score,Pred_Score,Mult_Factor
0,5.0,1.000000,0.769634,5.0
1,0.0,0.000000,0.474710,0.0
2,8.0,0.666667,0.728527,12.0
3,13.0,0.520000,0.757053,25.0
4,11.0,0.916667,0.711505,12.0
5,0.0,0.000000,0.882032,0.0
6,21.0,0.840000,0.773958,25.0
7,8.0,0.666667,0.815720,12.0
8,43.0,0.860000,0.747872,50.0
9,6.0,0.500000,0.672615,12.0


In [147]:
def denormalize(x):
    return np.around(x[2] * x[3])
cnn_df['Denorm_Pred_Score'] = cnn_df.apply(denormalize,axis=1)
cnn_df.head(10)

,Orig_Score,Norm_Score,Pred_Score,Mult_Factor,Denorm_Pred_Score
0,5.0,1.000000,0.769634,5.0,4.0
1,0.0,0.000000,0.474710,0.0,0.0
2,8.0,0.666667,0.728527,12.0,9.0
3,13.0,0.520000,0.757053,25.0,19.0
4,11.0,0.916667,0.711505,12.0,9.0
5,0.0,0.000000,0.882032,0.0,0.0
6,21.0,0.840000,0.773958,25.0,19.0
7,8.0,0.666667,0.815720,12.0,10.0
8,43.0,0.860000,0.747872,50.0,37.0
9,6.0,0.500000,0.672615,12.0,8.0


In [148]:
orig_score = cnn_df.Orig_Score.as_matrix(columns=None)
pred_score = cnn_df.Denorm_Pred_Score.as_matrix(columns=None)
rmse_cnn = RMSE(orig_score,pred_score)
print("RMSE: ",rmse_cnn)

RMSE:  2.916820824744241


In [149]:
c_kappa = CohenKappa(orig_score,pred_score)
print("Cohen Kappa: ",c_kappa)

Cohen Kappa:  0.25738861667132784


In [150]:
accuracy = accuracy_score(orig_score,pred_score)
print("Accuracy: ",accuracy)

Accuracy:  0.3219868954079552


In [104]:
# Old.. Ignore
prediction=estimator.predict(test_data)
rmse_val = RMSE(test_y_sentence,prediction)
print("RMSE: ",rmse_val)

RMSE:  0.16944170874087938


In [ ]:
estimator_lstm = KerasRegressor(build_fn=stack_lstm, epochs=20, batch_size=50)
estimator_lstm.fit(train_data, train_y_sentence)
prediction_lstm=estimator_lstm.predict(test_data)
rmse_val_lstm = RMSE(test_y_sentence,prediction_lstm)
print("RMSE: ",rmse_val_lstm)


Epoch 1/20
147120/147120 [==============================] - 409s 3ms/step - loss: 0.0301 - acc: 0.0991
Epoch 2/20
147120/147120 [==============================] - 393s 3ms/step - loss: 0.0281 - acc: 0.0999
Epoch 3/20
147120/147120 [==============================] - 383s 3ms/step - loss: 0.0272 - acc: 0.1004
Epoch 4/20
147120/147120 [==============================] - 379s 3ms/step - loss: 0.0264 - acc: 0.1009
Epoch 5/20
102000/147120 [===================>..........] - ETA: 1:57 - loss: 0.0256 - acc: 0.1012

In [113]:
cnn_df = pd.DataFrame([prediction.astype(np.double), prediction_lstm.astype(np.double), test_y_sentence.astype(np.double)]).transpose()
cnn_df.columns = ['CNN_Glove','Stacked LSTM','Actual']
cnn_df.head(10)

,CNN_Glove,Stacked LSTM,Actual
0,0.692931,0.723046,0.60
1,0.702375,0.678391,0.60
2,0.798613,0.734653,0.78
3,0.703292,0.677914,0.80
4,0.671944,0.669563,0.52
5,0.747734,0.820812,0.50
6,0.677340,0.650709,0.75
7,0.616736,0.506052,0.75
8,0.732711,0.706890,0.75
9,0.828496,0.792971,1.00


In [111]:
c_kappa = CohenKappa(test_y_sentence,cnn_glove_predict)
print("Cohen Kappa: ",c_kappa)

Cohen Kappa:  0.15593670766702683


In [114]:
c_kappa = CohenKappa(test_y_sentence,prediction_lstm)
print("Cohen Kappa LSTM: ",c_kappa)

Cohen Kappa LSTM:  0.14598100325900654


In [90]:

CNN_glove_model.fit(train_data,train_y_sentence, batch_size = 50, epochs=10, validation_split=0.15)

Train on 125052 samples, validate on 22068 samples
Epoch 1/10
125052/125052 [==============================] - 260s 2ms/step - loss: 0.0292 - acc: 0.0982 - val_loss: 0.0283 - val_acc: 0.1032
Epoch 2/10
125052/125052 [==============================] - 273s 2ms/step - loss: 0.0275 - acc: 0.0989 - val_loss: 0.0282 - val_acc: 0.1037
Epoch 3/10
125052/125052 [==============================] - 262s 2ms/step - loss: 0.0265 - acc: 0.0994 - val_loss: 0.0279 - val_acc: 0.1035
Epoch 4/10
125052/125052 [==============================] - 269s 2ms/step - loss: 0.0256 - acc: 0.1003 - val_loss: 0.0284 - val_acc: 0.1029
Epoch 5/10
125052/125052 [==============================] - 278s 2ms/step - loss: 0.0247 - acc: 0.1006 - val_loss: 0.0288 - val_acc: 0.1034
Epoch 6/10
125052/125052 [==============================] - 275s 2ms/step - loss: 0.0239 - acc: 0.1012 - val_loss: 0.0284 - val_acc: 0.1034
Epoch 7/10
125052/125052 [==============================] - 260s 2ms/step - loss: 0.0231 - acc: 0.1016 - val_

KeyboardInterrupt: 

In [70]:
cnn_glove_predict.shape

(36781, 1)

In [71]:
cnn_glove_predict = np.reshape(cnn_glove_predict,[cnn_glove_predict.shape[0],])
print(cnn_glove_predict.shape)
print(test_y_sentence.shape)

(36781,)
(36781,)


In [68]:
np.transpose(cnn_glove_predict).shape

(36781,)

In [74]:
CohenDF = pd.DataFrame([cnn_glove_predict.astype(np.double).round(), np.around(test_y_sentence.astype(np.double))]).transpose()
CohenDF.columns = ['actual','predict']
len(CohenDF)

36781

In [75]:
count = len(CohenDF)
CohenDF.columns = ['actual','predict']
correct = len(CohenDF[CohenDF.actual==CohenDF.predict])
acc = correct / count
print(acc)

0.8770289007911694


In [85]:
# If not rounded
CohenDF = pd.DataFrame([cnn_glove_predict.astype(np.double).round(), np.around(test_y_sentence.astype(np.double))]).transpose()
CohenDF.columns = ['actual','predict']
count = len(CohenDF)
CohenDF.columns = ['actual','predict']
correct = len(CohenDF[CohenDF.actual==CohenDF.predict])
acc = correct / count
print(acc)

0.8770289007911694


In [86]:
np.unique(CohenDF.actual,return_counts=True)

(array([0., 1.]), array([ 1138, 35643]))

In [87]:
CohenDF.head(30)

,actual,predict
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,0.0
6,1.0,1.0
7,1.0,1.0
8,1.0,1.0
9,1.0,1.0


In [42]:
rmse_val = RMSE(test_y_sentence,cnn_glove_predict)
print("RMSE: ",rmse_val)


RMSE:  0.17559793893398817


In [57]:
c_kappa = CohenKappa(test_y_sentence,cnn_glove_predict)
print("Cohen Kappa: ",c_kappa)

Cohen Kappa:  0.15593670766702683


In [66]:
cohen_kappa = cohen_kappa_score(test_y_sentence,cnn_glove_predict)
print("Cohen Kappa: ",cohen_kappa)

ValueError: continuous is not supported

## AESW

In [3]:
tree = ET.parse('../data-DNC/AESW/aesw2016_v1.2_train.xml')
root = tree.getroot()

In [4]:
sent_attrib = list()
sent_text = list()
for sent in root.iter('sentence'):
    sent_attrib.append(sent.attrib['sid'])
    sent_text.append(sent.text)

In [5]:
sent_dict = dict()
for sent in root.iter('sentence'):
    sid = sent.attrib['sid']
    sent_dict[sid]=sent.text

In [6]:
len(sent_dict)

1189412

### AESW Data for grammatical errors

In [74]:
aesw_train_labels = pd.read_csv("../data-DNC/AESW/aesw2016_v1.2_train.tok",sep='\t',encoding = "latin1",header=None)
aesw_train_labels.head(5)

,0,1,2
0,1,1.0,To facilitate an easier notation throughout t...
1,-1,1.0,To facilitate an easier notation throughout t...
2,0,1.1,Therefore MATH defines a special order of tim...
3,0,1.2,This is important since only MATH is the real...
4,0,1.3,Note that in all contour time-integrals we es...


In [75]:
aesw_train_labels.columns = ['Key_ID','Sent_ID','Text']
aesw_train_labels = aesw_train_labels[aesw_train_labels.Key_ID!=1]
len(aesw_train_labels)

1196903

In [76]:
mask = aesw_train_labels.Key_ID == -1
column_name = 'Key_ID'
aesw_train_labels.loc[mask, column_name] = 1

In [77]:
aesw_train_labels.head(5)

,Key_ID,Sent_ID,Text
1,1,1.0,To facilitate an easier notation throughout t...
2,0,1.1,Therefore MATH defines a special order of tim...
3,0,1.2,This is important since only MATH is the real...
4,0,1.3,Note that in all contour time-integrals we es...
5,0,2.0,Theorem REF proves the equivalence of ensembl...


In [78]:
x_train = aesw_train_labels.Text.as_matrix(columns=None)
y_train = aesw_train_labels.Key_ID.as_matrix(columns=None)

In [79]:
aesw_dev_labels = pd.read_csv("../data-DNC/AESW/aesw2016_v1.2_dev.tok",sep='\t',encoding = "latin1",header=None)
aesw_dev_labels.head(5)

,0,1,2
0,0,1.0,The team of robots in Chapter REF and CITE cr...
1,1,1.1,The Hamiltonian -LRB- Lyapunov function -RRB-...
2,-1,1.1,The Hamiltonian -LRB- Lyapunov function -RRB-...
3,1,1.2,"Therefore , for MATH robots , the time deriva..."
4,-1,1.2,"Therefore , for MATH robots the time derivati..."


In [80]:
aesw_dev_labels.columns = ['Key_ID','Sent_ID','Text']
aesw_dev_labels = aesw_dev_labels[aesw_dev_labels.Key_ID!=1]
len(aesw_dev_labels)

148409

In [81]:
mask = aesw_dev_labels.Key_ID == -1
column_name = 'Key_ID'
aesw_dev_labels.loc[mask, column_name] = 1

In [82]:
x_dev = aesw_dev_labels.Text.as_matrix(columns=None)
y_dev = aesw_dev_labels.Key_ID.as_matrix(columns=None)

In [83]:
vocabulary_size = 50000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(x_train)

In [91]:
train_seq = tokenizer.texts_to_sequences(x_train)
train_data = pad_sequences(train_seq)

In [93]:
max_seq_len = train_data.shape[1]
print(max_seq_len)

271


In [101]:
dev_seq = tokenizer.texts_to_sequences(x_dev)
dev_data = pad_sequences(dev_seq, maxlen=max_seq_len)
dev_data.shape

(148409, 271)

In [97]:
seed(2017)
conv = Sequential()
conv.add(Embedding(vocabulary_size, 100, input_length=max_seq_len))
conv.add(Conv1D(64, 5, activation = 'relu'))
conv.add(MaxPooling1D(2))
conv.add(Flatten())
conv.add(Dense(1, activation = 'sigmoid'))
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
conv.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])
conv.fit(train_data, y_train, batch_size = 500, epochs = 8, verbose = 0)

In [102]:
pred = conv.predict(dev_data)
pred_labels = (pred > 0.5).astype(np.int)
pred_labels = np.reshape(pred_labels,[dev_data.shape[0],])

#print(y_dev[0:5])
#print(pred_labels[0:5])
accuracy = accuracy_score(y_dev,pred_labels)
print("Accuracy: ",accuracy)

Accuracy:  0.607146466858479


In [58]:
print(np.shape(y_dev))
print(type(y_dev))
print(type(pred_labels))

(148409,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [36]:
embeddings_index = dict()
f = open('/Users/kurapati/W266/data/glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1917494 word vectors.


#### Using Glove word embeddings instead of learning

In [37]:
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [62]:
seed(2018)
conv_glove = Sequential()
conv_glove.add(Embedding(vocabulary_size, 300, input_length=50, weights=[embedding_matrix], trainable=False))
conv_glove.add(Conv1D(64, 5, activation = 'relu'))
conv_glove.add(MaxPooling1D(4))
conv_glove.add(Flatten())
conv_glove.add(Dense(1, activation = 'sigmoid'))
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
conv_glove.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])
conv_glove.fit(data, y_train, batch_size = 500, epochs = 8, verbose = 0)

In [63]:
pred = conv_glove.predict(dev_data)
pred_labels = (pred > 0.5).astype(np.int)
pred_labels = np.reshape(pred_labels,[148409,])

#print(y_dev[0:5])
#print(pred_labels[0:5])
accuracy = accuracy_score(y_dev,pred_labels)
print("Accuracy: ",accuracy)

Accuracy:  0.6500212251278561


In [65]:
# With CNN and LSTM
seed(2018)
conv_glove = Sequential()
conv_glove.add(Embedding(vocabulary_size, 300, input_length=50, weights=[embedding_matrix], trainable=False))
conv_glove.add(Conv1D(64, 5, activation = 'relu'))
conv_glove.add(MaxPooling1D(4))
conv_glove.add(LSTM(100))
conv_glove.add(Dense(1, activation = 'sigmoid'))
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
conv_glove.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics = ['accuracy'])
conv_glove.fit(data, y_train, batch_size = 500, epochs = 8, verbose = 0)

In [1]:
pred = conv_glove.predict(dev_data)
pred_labels = (pred > 0.5).astype(np.int)
pred_labels = np.reshape(pred_labels,[148409,])

#print(y_dev[0:5])
#print(pred_labels[0:5])
accuracy = accuracy_score(y_dev,pred_labels)
print("Accuracy: ",accuracy)

NameError: name 'conv_glove' is not defined

In [68]:
cnn_glove_f1 = f1_score(y_dev,pred_labels)
print("F1: ",cnn_glove_f1)

F1:  0.4894053721753443


In [70]:
average_precision = average_precision_score(y_dev,pred_labels)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.47


In [72]:
np.unique(y_dev,return_counts=True)

(array([0, 1]), array([90106, 58303]))

In [73]:
conv_glove.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 50, 300)           15000000  
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 46, 64)            96064     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 11, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 15,162,165
Trainable params: 162,165
Non-trainable params: 15,000,000
_________________________________________________________________


## TED

In [3]:
ted_main = pd.read_csv("../data-DNC/TED/ted_main.csv")
ted_transcript = pd.read_csv("../data-DNC/TED/transcripts.csv") 


In [3]:
ted_main.describe()

,comments,duration,film_date,languages,num_speaker,published_date,views
count,2550.000000,2550.000000,2.550000e+03,2550.000000,2550.000000,2.550000e+03,2.550000e+03
mean,191.562353,826.510196,1.321928e+09,27.326275,1.028235,1.343525e+09,1.698297e+06
std,282.315223,374.009138,1.197391e+08,9.563452,0.207705,9.464009e+07,2.498479e+06
min,2.000000,135.000000,7.464960e+07,0.000000,1.000000,1.151367e+09,5.044300e+04
25%,63.000000,577.000000,1.257466e+09,23.000000,1.000000,1.268463e+09,7.557928e+05
50%,118.000000,848.000000,1.333238e+09,28.000000,1.000000,1.340935e+09,1.124524e+06
75%,221.750000,1046.750000,1.412964e+09,33.000000,1.000000,1.423432e+09,1.700760e+06
max,6404.000000,5256.000000,1.503792e+09,72.000000,5.000000,1.506092e+09,4.722711e+07


In [4]:
ted_transcript.describe()

,transcript,url
count,2467,2467
unique,2464,2464
top,I'm going to tell you a little bit about my TE...,https://www.ted.com/talks/brene_brown_listenin...
freq,2,2


In [5]:
ted_main.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [6]:
ted_transcript.head(5)

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [17]:
print(len(ted_main))
print(len(ted_transcript))

2550
2467


### Not all TED MAIN has a transcript

In [24]:
ted_match = ted_main[ted_main.url.isin(ted_transcript.url)]
ted_notmatch = ted_main[~ted_main.url.isin(ted_transcript.url)]
len(ted_match)

2464

In [64]:
sentence = aeg_long.essay[0]
#sentence = ted_transcript.transcript[0]
s0 = nltk.word_tokenize(sentence)

In [65]:
t0 = Text(s0)

In [66]:
sentence_mod = ' '.join(s0)
print(sentence_mod)

Dear local newspaper , I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people , helps us learn about the globe ( astronomy ) and keeps us out of troble ! Thing about ! Dont you think so ? How would you feel if your teenager is always on the phone with friends ! Do you ever time to chat with your friends or buisness partner about things . Well now - there 's a new way to chat the computer , theirs plenty of sites on the internet to do so : @ ORGANIZATION1 , @ ORGANIZATION2 , @ CAPS1 , facebook , myspace ect . Just think now while your setting up meeting with your boss on the computer , your teenager is having fun on the phone not rushing to get off cause you want to use it . How did you learn about other countrys/states outside of yours ? Well I have by computer/internet , it 's a new way to learn about what going on in our time ! You might think your child spends a lot of time on the computer , but ask them s

In [67]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

lexical_diversity(t0)

0.4689119170984456

In [77]:
nlp = spacy.load('en_core_web_lg')

In [100]:
def most_similar(word):
    queries = [w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15]
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    return by_similarity[:4]

In [101]:
wordnet_lemmatizer = WordNetLemmatizer()
#word_syn_list = [wordnet_lemmatizer.lemmatize(w.lower_) for w in most_similar(nlp.vocab[u'Dear'])]
word_syn_list = [w.lower_ for w in most_similar(nlp.vocab[u'Safe'])]

In [102]:
set(word_syn_list)

{'safe', 'safety', 'secure'}

In [106]:
w_sample = nlp.vocab[u'Safe']
print(type(w_sample))
for w in w_sample.vocab:
    print(w.prob)

<class 'spacy.lexeme.Lexeme'>


In [68]:
doc = nlp(sentence_mod)

In [69]:
spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in doc]

In [70]:
df_spacy = pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])
nltk_pos_tagged = nltk.pos_tag(sentence.split())
df_nltk = pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])


In [71]:
df_spacy

,Word,POS tag,Tag type
0,Dear,JJ,ADJ
1,local,JJ,ADJ
2,newspaper,NN,NOUN
3,",",",",PUNCT
4,I,PRP,PRON
5,think,VBP,VERB
6,effects,NNS,NOUN
7,computers,NNS,NOUN
8,have,VBP,VERB
9,on,IN,ADP


In [72]:
df_nltk

,Word,POS tag
0,Dear,NNP
1,local,JJ
2,"newspaper,",NN
3,I,PRP
4,think,VBP
5,effects,NNS
6,computers,NNS
7,have,VBP
8,on,IN
9,people,NNS


In [73]:
for entity in doc.ents:
    print(entity.text, entity.label_)

ORGANIZATION2 GPE
DATE1 ORG


In [111]:
stop_words = set(stopwords.words('english'))
word_syns = defaultdict(lambda: set())
for word_index,word in enumerate(s0):
    if word not in stop_words:
        word_pos = doc[word_index].pos_
        if word_pos == "ADJ":
            syns = wordnet.synsets(word,wordnet.ADJ)
        elif word_pos == "NOUN" or "PROPN":
            syns = wordnet.synsets(word,wordnet.NOUN)
        elif word_pos == "ADV":
            syns = wordnet.synsets(word,wordnet.ADV)
        else:
            continue
            #if word not in stop_words:
            #or word_pos == "NOUN" or word_pos == "VERB"
        for i in range(len(syns)):
            word_syns[word].add(syns[i].lemmas()[0].name())

print(word_syns)


defaultdict(<function <lambda> at 0x1a42ac9a60>, {'Dear': {'costly', 'beloved', 'dear'}, 'local': {'local', 'local_anesthetic'}, 'newspaper': {'newspaper'}, 'I': {'I', 'iodine', 'one'}, 'think': {'think'}, 'effects': {'effects', 'effect', 'consequence', 'impression'}, 'computers': {'computer', 'calculator'}, 'people': {'citizenry', 'people', 'multitude'}, 'great': {'bang-up', 'big', 'great', 'capital'}, 'learning': {'eruditeness', 'learning'}, 'give': {'give'}, 'us': {'U', 'United_States', 'uracil', 'uranium'}, 'time': {'clock_time', 'prison_term', 'time', 'meter', 'fourth_dimension'}, 'chat': {'New_World_chat', 'Old_World_chat', 'chat'}, 'helps': {'avail', 'assistant', 'aid'}, 'globe': {'Earth', 'globe', 'ball'}, 'astronomy': {'astronomy'}, 'keeps': {'support', 'keep', 'hold'}, 'Thing': {'matter', 'thing'}, 'feel': {'feel', 'spirit', 'tactile_property'}, 'teenager': {'adolescent'}, 'phone': {'telephone', 'phone', 'earphone'}, 'friends': {'ally', 'friend', 'Friend', 'acquaintance', 'su

In [112]:
# Use spacy instead of wordnet
word_syns_spacy = defaultdict(lambda: set())
for word_index,word in enumerate(s0):
    if word not in stop_words:
        word_pos = doc[word_index].pos_
        if word_pos == "ADJ" or word_pos == "NOUN" or word_pos == "PROPN" or word_pos == "ADV":
            word_syn_list = [w.lower_ for w in most_similar(nlp.vocab[word])]
            for syn_w in word_syn_list:
                word_syns_spacy[word].add(syn_w)

print(word_syns_spacy)

defaultdict(<function <lambda> at 0x1a42d03e18>, {'Dear': {'thank', 'dear'}, 'local': {'local', 'businesses', 'locally', 'regional'}, 'newspaper': {'tabloid', 'newspapers', 'magazine', 'newspaper'}, 'effects': {'effects', 'adverse', 'affect', 'effect'}, 'computers': {'laptops', 'computers', 'desktops', 'computer'}, 'people': {'individuals', 'those', 'people', 'others'}, 'great': {'fantastic', 'good', 'great', 'wonderful'}, 'learning': {'learn', 'teach', 'teaching', 'learning'}, 'skills/affects': {'educated', 'wade', 'eradicate', 'equator'}, 'chat': {'chatroom', 'chatting', 'chat', 'chats'}, ',': {'and', ',', ';'}, 'us': {'our', 'us', 'we', 'them'}, 'learn': {'understand', 'teach', 'learn', 'learning'}, '!': {':)', '!', ':-)', ';)'}, 'How': {'what', 'how'}, 'feel': {'felt', 'feels', 'feel', 'feeling'}, 'phone': {'phone', 'cellphone', 'phones', 'telephone'}, 'friends': {'buddies', 'pals', 'friend', 'friends'}, 'time': {'day', 'time', 'once', 'when'}, 'things': {'things', 'everything', 's

In [110]:
wordnet.synsets('safe',wordnet.NOUN)

[Synset('safe.n.01'), Synset('safe.n.02'), Synset('condom.n.01')]